# Notebook : Why Rag

# About

In this notebook, we will motivate Retrieval Augmented Generation (RAG) .

We will see why we can't use Large Language Model directly.



# Imports

In [1]:
from dotenv import load_dotenv
import rich
import logging
from llama_index.core.llms import ChatMessage
from llama_index.llms.openai import OpenAI
from huggingface_hub import InferenceClient
import os

In [2]:
#logging.basicConfig(level=logging.DEBUG)


In [3]:
load_dotenv(dotenv_path="../env")

True

In [4]:
llm = OpenAI(model="gpt-3.5-turbo",temperature=0)


# Problems

There are atleast two problems with using Large Language Model 

- Knoweldge Cutoff
- Hallucination

## Knoweldge Cutoff

In [5]:
llm = OpenAI(model="gpt-4o-mini",temperature=0)


In [6]:
def get_response(query:str):
    messages = [
        ChatMessage(role="user", content=query),
    ]
    
    resp = llm.chat(messages)

    return resp

In [7]:
query="what is different about Llama3.2 than Llama2 ?"


In [8]:
response = get_response(query)

In [9]:
rich.print(response.message.content)

As of my last update in October 2023, specific details about Llama 3.2 compared to Llama 2 have not been officially
released. However, in general, when comparing iterations of AI models like Llama, you can expect improvements in 
several areas:

1. **Model Architecture**: Newer versions often incorporate advancements in model architecture that enhance 
performance, efficiency, and scalability.

2. **Training Data**: Llama 3.2 may be trained on a larger and more diverse dataset, which can improve its 
understanding of language and context.

3. **Performance**: Improvements in natural language understanding, generation quality, and response coherence are 
common in newer models.

4. **Fine-tuning and Adaptability**: Enhanced capabilities for fine-tuning on specific tasks or domains may be 
introduced, allowing for better customization.

5. **Safety and Ethical Considerations**: Newer models often include better mechanisms for handling harmful 
content, bias reduction, and overall ethical AI use.

6. **Efficiency**: Optimizations that reduce computational requirements while maintaining or improving performance 
are often a focus in newer versions.

For the most accurate and detailed information, I recommend checking the official announcements or documentation 
from the developers of the Llama models.

In [11]:
rich.print(response)

ChatResponse(
    message=ChatMessage(
        role=<MessageRole.ASSISTANT: 'assistant'>,
        content='As of my last update in October 2023, specific details about Llama 3.2 compared to Llama 2 have 
not been officially released. However, in general, when comparing iterations of AI models like Llama, you can 
expect improvements in several areas:\n\n1. **Model Architecture**: Newer versions often incorporate advancements 
in model architecture that enhance performance, efficiency, and scalability.\n\n2. **Training Data**: Llama 3.2 may
be trained on a larger and more diverse dataset, which can improve its understanding of language and context.\n\n3.
**Performance**: Improvements in natural language understanding, generation quality, and response coherence are 
common in newer models.\n\n4. **Fine-tuning and Adaptability**: Enhanced capabilities for fine-tuning on specific 
tasks or domains may be introduced, allowing for better customization.\n\n5. **Safety and Ethical Considerations**:
Newer models often include better mechanisms for handling harmful content, bias reduction, and overall ethical AI 
use.\n\n6. **Efficiency**: Optimizations that reduce computational requirements while maintaining or improving 
performance are often a focus in newer versions.\n\nFor the most accurate and detailed information, I recommend 
checking the official announcements or documentation from the developers of the Llama models.',
        additional_kwargs={}
    ),
    raw=ChatCompletion(
        id='chatcmpl-APXHJVVHV9JHNwiuJ6rNAgtyEQSsT',
        choices=[
            Choice(
                finish_reason='stop',
                index=0,
                logprobs=None,
                message=ChatCompletionMessage(
                    content='As of my last update in October 2023, specific details about Llama 3.2 compared to 
Llama 2 have not been officially released. However, in general, when comparing iterations of AI models like Llama, 
you can expect improvements in several areas:\n\n1. **Model Architecture**: Newer versions often incorporate 
advancements in model architecture that enhance performance, efficiency, and scalability.\n\n2. **Training Data**: 
Llama 3.2 may be trained on a larger and more diverse dataset, which can improve its understanding of language and 
context.\n\n3. **Performance**: Improvements in natural language understanding, generation quality, and response 
coherence are common in newer models.\n\n4. **Fine-tuning and Adaptability**: Enhanced capabilities for fine-tuning
on specific tasks or domains may be introduced, allowing for better customization.\n\n5. **Safety and Ethical 
Considerations**: Newer models often include better mechanisms for handling harmful content, bias reduction, and 
overall ethical AI use.\n\n6. **Efficiency**: Optimizations that reduce computational requirements while 
maintaining or improving performance are often a focus in newer versions.\n\nFor the most accurate and detailed 
information, I recommend checking the official announcements or documentation from the developers of the Llama 
models.',
                    refusal=None,
                    role='assistant',
                    audio=None,
                    function_call=None,
                    tool_calls=None
                )
            )
        ],
        created=1730649485,
        model='gpt-4o-mini-2024-07-18',
        object='chat.completion',
        service_tier=None,
        system_fingerprint='fp_0ba0d124f1',
        usage=CompletionUsage(
            completion_tokens=250,
            prompt_tokens=21,
            total_tokens=271,
            completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0),
            prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0)
        )
    ),
    delta=None,
    logprobs=None,
    additional_kwargs={'prompt_tokens': 21, 'completion_tokens': 250, 'total_tokens': 271}
)

note the model is aliased to `model='gpt-4o-mini-2024-07-18'`.

So, the model's training data wont have any information after then.

## Hallucination

![Snapshot of Hallucination](../images/llama_2__hallucination.png)


In the above video, we ask about non existant Llama model.

We also give it a link to a study music youtube video.

Note: we are also using an "older" version of Llama model to make the inference

In [12]:
client = InferenceClient(api_key=os.environ['HF_API_KEY'])

In [13]:
message = f"""
What did Andrej Karpathy say about Meta's Llama 5.9 in the below youtube talk

https://www.youtube.com/watch?v=n61ULEU7CO0&ab_channel=LofiGirl

"""

In [14]:
messages = [
    { "role": "user", "content": message },
]

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf", 
    messages=messages, 
    temperature=0,
)

In [15]:
rich.print(response)

ChatCompletionOutput(
    choices=[
        ChatCompletionOutputComplete(
            finish_reason='length',
            index=0,
            message=ChatCompletionOutputMessage(
                role='assistant',
                content="  Andrej Karpathy, a well-known computer scientist and researcher, did not make any 
comments about Meta's Llama 5.9 in the YouTube talk you provided. The talk is actually about the use of language 
models in music generation, and Karpathy's comments are focused on the topic of language models and their 
applications in various fields.\n\nTherefore, I cannot provide any information about what Karpathy said about 
Meta's Llama",
                tool_calls=None
            ),
            logprobs=None
        )
    ],
    created=1730649490,
    id='',
    model='meta-llama/Llama-2-7b-chat-hf',
    system_fingerprint='2.3.1-dev0-sha-169178b',
    usage=ChatCompletionOutputUsage(completion_tokens=100, prompt_tokens=67, total_tokens=167)
)

In [16]:
rich.print(response.choices[0].message.content)

Andrej Karpathy, a well-known computer scientist and researcher, did not make any comments about Meta's Llama 5.9
in the YouTube talk you provided. The talk is actually about the use of language models in music generation, and 
Karpathy's comments are focused on the topic of language models and their applications in various fields.

Therefore, I cannot provide any information about what Karpathy said about Meta's Llama

## Notes

We explored at least two problems with directly using LLMs directly. 

- Knowledge Cuttoff  
- Hallucination.

